## Import Libraries

In [ ]:
!pip install opendatasets
!pip install pandas

you can use my kaggle credentials:

nomielagarde

d8e0f8ad13e663040a0d037b2208f1b6

In [ ]:
import opendatasets as od
import pandas as pd

od.download("https://www.kaggle.com/datasets/nomielagarde/huron-dataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: nomielagarde
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/nomielagarde/huron-dataset


100%|██████████| 3.35G/3.35G [02:33<00:00, 23.4MB/s]


In [ ]:
#installing smp libraru
!pip install segmentation-models-pytorch

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 87.9 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=1dd7668e2c0f9c14e7ed601002c8fdcfd6f21df9cfaa3bcc371f7fc54f46eea6
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60944 sha256=6ff96b13a821c4d9f85eca9a0c020ca64fd946e390491d4107978004f405439d
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built efficientnet-pytorc

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import os
from pathlib import Path
import numpy as np
from tqdm import tqdm
import segmentation_models_pytorch as smp
import matplotlib.pyplot as plt
import random
import csv
import pandas as pd

## Classes and functions

### Dataset

In [ ]:
# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


class HuronDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = Path(image_dir)
        self.mask_dir = Path(mask_dir)
        self.transform = transform
        self.mask_transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor()
        ])
        self.images = sorted(os.listdir(image_dir))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.image_dir / self.images[idx]
        mask_path = self.mask_dir / self.images[idx]

        image = Image.open(img_path).convert('RGB')
        mask = Image.open(mask_path).convert('L')

        if self.transform:
            image = self.transform(image)
        mask = self.mask_transform(mask)

        # Ensure mask is binary
        mask = (mask > 0.5).float()

        return image, mask

### Eval and Metrics

In [ ]:
def calculate_iou(outputs, targets, threshold=0.5):
    outputs = (outputs > threshold).float()
    targets = targets.float()

    intersection = (outputs * targets).sum(dim=(1, 2))
    union = outputs.sum(dim=(1, 2)) + targets.sum(dim=(1, 2)) - intersection

    iou = (intersection + 1e-6) / (union + 1e-6)
    return iou.mean()

def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    total_iou = 0
    num_batches = len(dataloader)

    with torch.no_grad():
        for images, masks in dataloader:
            images = images.to(device)
            masks = masks.to(device)

            outputs = model(images)
            loss = criterion(outputs, masks)
            iou = calculate_iou(outputs, masks)

            total_loss += loss.item()
            total_iou += iou.item()

    return total_loss / num_batches, total_iou / num_batches

def reset_weights(m):
    for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            layer.reset_parameters()

class CombinedLoss(nn.Module):
    def __init__(self, loss1, loss2, weight1=1.0, weight2=1.0):
        super(CombinedLoss, self).__init__()
        self.loss1 = loss1
        self.loss2 = loss2
        self.weight1 = weight1
        self.weight2 = weight2

    def forward(self, y_pred, y_true):
        return self.weight1 * self.loss1(y_pred, y_true) + self.weight2 * self.loss2(y_pred, y_true)

### Training function

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=5, model_filename='best_model.pth'):
    best_val_iou = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_iou = 0.0

        with tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}') as pbar:
            for images, masks in pbar:
                images = images.to(device)
                masks = masks.to(device)

                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, masks)
                iou = calculate_iou(outputs, masks)

                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                running_iou += iou.item()

                pbar.set_postfix({
                    'loss': running_loss / (pbar.n + 1),
                    'IoU': running_iou / (pbar.n + 1)
                })

        # Validate
        model.eval()
        val_loss, val_iou = evaluate(model, val_loader, criterion, device)
        print(f'Epoch {epoch+1}/{num_epochs} - Validation Loss: {val_loss:.4f}, Validation IoU: {val_iou:.4f}')

        if val_iou > best_val_iou:
            best_val_iou = val_iou
            torch.save(model.state_dict(), model_filename)

## Defining models and loss functions

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Dataset paths
image_dir = '/content/huron-dataset/Sliced_Images'
mask_dir = '/content/huron-dataset/Sliced_masks'

# Transform for input images
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                       std=[0.229, 0.224, 0.225])
])

# Create full dataset
full_dataset = HuronDataset(image_dir, mask_dir, transform=transform)

# Calculate splits
total_size = len(full_dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

# Split dataset
train_dataset, val_dataset, test_dataset = random_split(
    full_dataset,
    [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

print(f"Dataset splits: Train={len(train_dataset)}, Val={len(val_dataset)}, Test={len(test_dataset)}")

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=4)

# Define model configurations
model_classes = [
    {
        'name': 'U-Net++',
        'class': smp.UnetPlusPlus,
        'args': {}
    },
    {
        'name': 'U-Net++ with SCSE',
        'class': smp.UnetPlusPlus,
        'args': {'decoder_attention_type': 'scse'}
    },
    {
        'name': 'FPN',
        'class': smp.FPN,
        'args': {}
    },
    {
        'name': 'PAN',
        'class': smp.PAN,
        'args': {'decoder_channels': 256}
    }
]

# Define loss functions
loss_functions = [
    {'name': 'DiceLoss', 'function': smp.losses.DiceLoss(mode='binary')},
    {'name': 'TverskyLoss', 'function': smp.losses.TverskyLoss(mode='binary')},
    {'name': 'FocalLoss', 'function': smp.losses.FocalLoss(mode='binary')},
    {'name': 'BCEWithLogitsLoss', 'function': nn.BCEWithLogitsLoss()},
    {'name': 'DiceLoss_plus_FocalLoss', 'function': CombinedLoss(
        smp.losses.DiceLoss(mode='binary'),
        smp.losses.FocalLoss(mode='binary')
    )},
    {'name': 'DiceLoss_plus_BCEWithLogitsLoss', 'function': CombinedLoss(
        smp.losses.DiceLoss(mode='binary'),
        nn.BCEWithLogitsLoss()
    )},
]

Using device: cuda
Dataset splits: Train=12142, Val=2602, Test=2603


## Encoder 1: resnext50_32x4d

In [ ]:
encoder_name = 'resnext50_32x4d'
results = []
print(f"Training with {encoder_name} encoder")
# Iterate over models
for model_info in model_classes:
    model_class = model_info['class']
    model_args = model_info['args']
    model_name = model_info['name']
    # Iterate over loss functions
    for loss_func_info in loss_functions:
        loss_func_name = loss_func_info['name']
        loss_function = loss_func_info['function']
        model_filename = f"{model_name}_{encoder_name}_{loss_func_name}.pth".replace(' ', '_').replace('+', 'plus')

        # Skip if model file already exists
        if os.path.exists(model_filename):
            print(f"Skipping {model_filename} - already exists")
            continue
        # Define the model
        model = model_class(
            encoder_name=encoder_name,
            encoder_weights="imagenet",
            in_channels=3,
            classes=1,
            **model_args
        ).to(device)
        # Reset model weights
        model.apply(reset_weights)
        # Define loss function and optimizer
        criterion = loss_function
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        # Train the model
        print(f"Training {model_name} with {encoder_name} encoder and {loss_func_name}.")
        train_model(
            model, train_loader, val_loader, criterion, optimizer, device,
            num_epochs=5, model_filename=model_filename
        )
        # Load the best model and evaluate on test set
        model.load_state_dict(torch.load(model_filename))
        test_loss, test_iou = evaluate(model, test_loader, criterion, device)

        # Print test metrics
        print(f"\nTest Results for {model_name} with {loss_func_name}:")
        print(f"Test Loss: {test_loss:.4f}")
        print(f"Test IoU: {test_iou:.4f}\n")

        # Log results
        results.append({
            'Encoder': encoder_name,
            'Model': model_name,
            'Loss Function': loss_func_name,
            'Epochs': 5,
            'Test Loss': test_loss,    # Changed from Validation Loss
            'Test IoU': test_iou,      # Changed from Validation IoU
            'Model Filename': model_filename
        })
        # Save interim results
        interim_df = pd.DataFrame(results)
        interim_df.to_csv(f'results_{encoder_name}.csv', index=False)
print(f"Completed training with {encoder_name}")

Training with resnext50_32x4d encoder


Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth
100%|██████████| 95.8M/95.8M [00:00<00:00, 436MB/s]


Training U-Net++ with resnext50_32x4d encoder and DiceLoss.


Epoch 1/5: 100%|██████████| 1518/1518 [02:16<00:00, 11.10it/s, loss=0.139, IoU=0.858]


Epoch 1/5 - Validation Loss: 0.6296, Validation IoU: 0.3642


...

Epoch 3/5 - Validation Loss: 0.1711, Validation IoU: 0.8981


Epoch 4/5: 100%|██████████| 1518/1518 [01:09<00:00, 21.83it/s, loss=0.189, IoU=0.894]


Epoch 4/5 - Validation Loss: 0.1789, Validation IoU: 0.8987


Epoch 5/5: 100%|██████████| 1518/1518 [01:09<00:00, 21.76it/s, loss=0.183, IoU=0.897]


Epoch 5/5 - Validation Loss: 0.1843, Validation IoU: 0.9007

Test Results for FPN with DiceLoss_plus_BCEWithLogitsLoss:
Test Loss: 0.1936
Test IoU: 0.8986

Completed training with resnext50_32x4d


## Encoder 2: resnet34

In [ ]:
encoder_name = 'resnet34'
results = []
print(f"Training with {encoder_name} encoder")
# Iterate over models
for model_info in model_classes:
    model_class = model_info['class']
    model_args = model_info['args']
    model_name = model_info['name']
    # Iterate over loss functions
    for loss_func_info in loss_functions:
        loss_func_name = loss_func_info['name']
        loss_function = loss_func_info['function']
        model_filename = f"{model_name}_{encoder_name}_{loss_func_name}.pth".replace(' ', '_').replace('+', 'plus')

        # Skip if model file already exists
        if os.path.exists(model_filename):
            print(f"Skipping {model_filename} - already exists")
            continue
        # Define the model
        model = model_class(
            encoder_name=encoder_name,
            encoder_weights="imagenet",
            in_channels=3,
            classes=1,
            **model_args
        ).to(device)
        # Reset model weights
        model.apply(reset_weights)
        # Define loss function and optimizer
        criterion = loss_function
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        # Train the model
        print(f"Training {model_name} with {encoder_name} encoder and {loss_func_name}.")
        train_model(
            model, train_loader, val_loader, criterion, optimizer, device,
            num_epochs=5, model_filename=model_filename
        )
        # Load the best model and evaluate on test set
        model.load_state_dict(torch.load(model_filename))
        test_loss, test_iou = evaluate(model, test_loader, criterion, device)

        # Print test metrics
        print(f"\nTest Results for {model_name} with {loss_func_name}:")
        print(f"Test Loss: {test_loss:.4f}")
        print(f"Test IoU: {test_iou:.4f}\n")

        # Log results
        results.append({
            'Encoder': encoder_name,
            'Model': model_name,
            'Loss Function': loss_func_name,
            'Epochs': 5,
            'Test Loss': test_loss,    # Changed from Validation Loss
            'Test IoU': test_iou,      # Changed from Validation IoU
            'Model Filename': model_filename
        })
        # Save interim results
        interim_df = pd.DataFrame(results)
        interim_df.to_csv(f'results_{encoder_name}.csv', index=False)
print(f"Completed training with {encoder_name}")

Training with resnet34 encoder


Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 268MB/s]


Training U-Net++ with resnet34 encoder and DiceLoss.


Epoch 1/5: 100%|██████████| 1518/1518 [01:27<00:00, 17.45it/s, loss=0.147, IoU=0.849]


Epoch 1/5 - Validation Loss: 0.6294, Validation IoU: 0.3439


...

Epoch 3/5 - Validation Loss: 0.3178, Validation IoU: 0.8571


Epoch 4/5: 100%|██████████| 1518/1518 [01:01<00:00, 24.69it/s, loss=0.201, IoU=0.891]


Epoch 4/5 - Validation Loss: 0.1936, Validation IoU: 0.8925


Epoch 5/5: 100%|██████████| 1518/1518 [01:00<00:00, 25.24it/s, loss=0.202, IoU=0.89]


Epoch 5/5 - Validation Loss: 0.1812, Validation IoU: 0.8925

Test Results for FPN with DiceLoss_plus_BCEWithLogitsLoss:
Test Loss: 0.1859
Test IoU: 0.8914

Completed training with resnet34


## Encoder 3: efficientnet-b4

In [ ]:
encoder_name = 'efficientnet-b4'
results = []
print(f"Training with {encoder_name} encoder")
# Iterate over models
for model_info in model_classes:
    model_class = model_info['class']
    model_args = model_info['args']
    model_name = model_info['name']
    # Iterate over loss functions
    for loss_func_info in loss_functions:
        loss_func_name = loss_func_info['name']
        loss_function = loss_func_info['function']
        model_filename = f"{model_name}_{encoder_name}_{loss_func_name}.pth".replace(' ', '_').replace('+', 'plus')

        # Skip if model file already exists
        if os.path.exists(model_filename):
            print(f"Skipping {model_filename} - already exists")
            continue
        # Define the model
        model = model_class(
            encoder_name=encoder_name,
            encoder_weights="imagenet",
            in_channels=3,
            classes=1,
            **model_args
        ).to(device)
        # Reset model weights
        model.apply(reset_weights)
        # Define loss function and optimizer
        criterion = loss_function
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        # Train the model
        print(f"Training {model_name} with {encoder_name} encoder and {loss_func_name}.")
        train_model(
            model, train_loader, val_loader, criterion, optimizer, device,
            num_epochs=5, model_filename=model_filename
        )
        # Load the best model and evaluate on test set
        model.load_state_dict(torch.load(model_filename))
        test_loss, test_iou = evaluate(model, test_loader, criterion, device)

        # Print test metrics
        print(f"\nTest Results for {model_name} with {loss_func_name}:")
        print(f"Test Loss: {test_loss:.4f}")
        print(f"Test IoU: {test_iou:.4f}\n")

        # Log results
        results.append({
            'Encoder': encoder_name,
            'Model': model_name,
            'Loss Function': loss_func_name,
            'Epochs': 5,
            'Test Loss': test_loss,    # Changed from Validation Loss
            'Test IoU': test_iou,      # Changed from Validation IoU
            'Model Filename': model_filename
        })
        # Save interim results
        interim_df = pd.DataFrame(results)
        interim_df.to_csv(f'results_{encoder_name}.csv', index=False)
print(f"Completed training with {encoder_name}")

Training with efficientnet-b4 encoder
Training U-Net++ with efficientnet-b4 encoder and DiceLoss.


Epoch 1/5: 100%|██████████| 1518/1518 [02:53<00:00,  8.75it/s, loss=0.117, IoU=0.873]


Epoch 1/5 - Validation Loss: 0.0969, Validation IoU: 0.8866


Epoch 2/5: 100%|██████████| 1518/1518 [02:55<00:00,  8.63it/s, loss=0.0843, IoU=0.898]


Epoch 2/5 - Validation Loss: 0.0836, Validation IoU: 0.8987


...

Epoch 3/5 - Validation Loss: 0.1603, Validation IoU: 0.9016


Epoch 4/5: 100%|██████████| 1518/1518 [02:46<00:00,  9.12it/s, loss=0.18, IoU=0.9]


Epoch 4/5 - Validation Loss: 0.1953, Validation IoU: 0.8963


Epoch 5/5: 100%|██████████| 1518/1518 [02:46<00:00,  9.11it/s, loss=0.17, IoU=0.903]


Epoch 5/5 - Validation Loss: 0.2461, Validation IoU: 0.8718

Test Results for FPN with DiceLoss_plus_BCEWithLogitsLoss:
Test Loss: 0.1674
Test IoU: 0.9001

Completed training with efficientnet-b4


## Encoder 4: se_resnext50_32x4d

In [ ]:
from google.colab import files
files.download('results_efficientnet-b4.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import ssl

try:
  _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
  pass
else:
  ssl._create_default_https_context = _create_unverified_https_context

In [ ]:
encoder_name = 'se_resnext50_32x4d'
results = []
print(f"Training with {encoder_name} encoder")
# Iterate over models
for model_info in model_classes:
    model_class = model_info['class']
    model_args = model_info['args']
    model_name = model_info['name']
    # Iterate over loss functions
    for loss_func_info in loss_functions:
        loss_func_name = loss_func_info['name']
        loss_function = loss_func_info['function']
        model_filename = f"{model_name}_{encoder_name}_{loss_func_name}.pth".replace(' ', '_').replace('+', 'plus')

        # Skip if model file already exists
        if os.path.exists(model_filename):
            print(f"Skipping {model_filename} - already exists")
            continue
        # Define the model
        model = model_class(
            encoder_name=encoder_name,
            encoder_weights="imagenet",
            in_channels=3,
            classes=1,
            **model_args
        ).to(device)
        # Reset model weights
        model.apply(reset_weights)
        # Define loss function and optimizer
        criterion = loss_function
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        # Train the model
        print(f"Training {model_name} with {encoder_name} encoder and {loss_func_name}.")
        train_model(
            model, train_loader, val_loader, criterion, optimizer, device,
            num_epochs=5, model_filename=model_filename
        )
        # Load the best model and evaluate on test set
        model.load_state_dict(torch.load(model_filename))
        test_loss, test_iou = evaluate(model, test_loader, criterion, device)

        # Print test metrics
        print(f"\nTest Results for {model_name} with {loss_func_name}:")
        print(f"Test Loss: {test_loss:.4f}")
        print(f"Test IoU: {test_iou:.4f}\n")

        # Log results
        results.append({
            'Encoder': encoder_name,
            'Model': model_name,
            'Loss Function': loss_func_name,
            'Epochs': 5,
            'Test Loss': test_loss,    # Changed from Validation Loss
            'Test IoU': test_iou,      # Changed from Validation IoU
            'Model Filename': model_filename
        })
        # Save interim results
        interim_df = pd.DataFrame(results)
        interim_df.to_csv(f'results_{encoder_name}.csv', index=False)
print(f"Completed training with {encoder_name}")

Training with se_resnext50_32x4d encoder


Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/se_resnext50_32x4d-a260b3a4.pth" to /root/.cache/torch/hub/checkpoints/se_resnext50_32x4d-a260b3a4.pth
100%|██████████| 105M/105M [05:10<00:00, 356kB/s]


Training U-Net++ with se_resnext50_32x4d encoder and DiceLoss.


Epoch 1/5: 100%|██████████| 1518/1518 [02:27<00:00, 10.27it/s, loss=0.133, IoU=0.864]


Epoch 1/5 - Validation Loss: 0.1207, Validation IoU: 0.8686


...

Epoch 5/5 - Validation Loss: 2.4701, Validation IoU: 0.3671

Test Results for FPN with DiceLoss_plus_BCEWithLogitsLoss:
Test Loss: 0.1798
Test IoU: 0.8962

Training PAN with se_resnext50_32x4d encoder and DiceLoss.


Epoch 1/5: 100%|██████████| 1518/1518 [13:27<00:00,  1.88it/s, loss=0.128, IoU=0.859]


Epoch 1/5 - Validation Loss: 0.1053, Validation IoU: 0.8721


Epoch 2/5:   3%|▎         | 50/1518 [00:27<13:16,  1.84it/s, loss=0.0888, IoU=0.883]


KeyboardInterrupt: 